In [1]:
# -*- coding: utf-8 -*-
import pickle
import tensorflow as tf
import numpy as np
# import sklearn

from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import HoverTool, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
estate_file = open('./data/estate_suji.dat','rb')
estate_price = pickle.load(estate_file)
estate_price_cnt = len(estate_price)
chart_name = "# 170310, 수지구 아파트 매매가 {0}건 (NAVER)".format(str(estate_price_cnt))

before_x = []
before_y = []

names = []
labels = []

print(estate_price[0])

for item in estate_price:
    p = item['price']/10000
    before_x.append(item['space'])
    before_y.append(p)
    names.append(item['name'])
    labels.append("{}평/{}억".format(item['space'], p))

# sklearn.preprocessing.minmax_scale()

# before_x = np.array(before_x)
# before_y = np.array(before_y)
# print(before_x.shape)
# print(before_y.shape)

{'name': '만현마을5단지현대아이파크7', 'price': 37000, 'space': 37}


In [3]:
# tf.Variable: 그래프를 계산하면서 최적화 할 변수들입니다. 
# 이 값이 바로 신경망을 좌우하는 값들입니다.

# tf.random_uniform: 균일 한 분포에서 임의의 값을 출력합니다.
# tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)
X = tf.placeholder(tf.float32, name='area')
Y = tf.placeholder(tf.float32, name='price')

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='Weight')
# W = tf.get_variable('w', [1], initializer=tf.contrib.layers.xavier_initializer())
B = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='b')

In [4]:
hypothesis = W*X+B

# 손실 함수를 작성합니다.
# mean(h - Y)^2 : 예측값과 실제값의 거리를 비용(손실) 함수로 정합니다.
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [5]:
# 텐서플로우에 기본적으로 포함되어 있는 함수를 이용해 경사 하강법 최적화를 수행합니다.
train = tf.train.GradientDescentOptimizer(learning_rate=0.0000005).minimize(cost)
# train = tf.train.AdamOptimizer(0.1).minimize(cost)

In [6]:
history = {
    'cost' : [],
    'hypothesis' : [],
    'weight': [],
}

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(5000):
    # sess.run 을 통해 train_op 와 cost 그래프를 계산합니다.
    # 이 때, 가설 수식에 넣어야 할 실제값을 feed_dict 을 통해 전달합니다.
    _, c, w, b = sess.run([train, cost, W, B], feed_dict={X: before_x, Y: before_y})

    if step % 100 == 0:
        history['cost'].append(c)
        history['weight'].append(w)
        
        print("step: {0}, cost: {1}, w: {2}, b: {3}".format(step, c, w, b))

step: 0, cost: 600.333984375, w: [-0.4841345], b: [-0.21683757]
step: 100, cost: 431.2950134277344, w: [-0.39244434], b: [-0.21467789]
step: 200, cost: 309.9561462402344, w: [-0.31476104], b: [-0.21284488]
step: 300, cost: 222.85797119140625, w: [-0.24894506], b: [-0.21128878]
step: 400, cost: 160.33779907226562, w: [-0.19318326], b: [-0.20996714]
step: 500, cost: 115.46002960205078, w: [-0.14593983], b: [-0.20884424]
step: 600, cost: 83.24626922607422, w: [-0.10591345], b: [-0.20788975]
step: 700, cost: 60.122764587402344, w: [-0.07200167], b: [-0.20707783]
step: 800, cost: 43.52444076538086, w: [-0.04327036], b: [-0.20638674]
step: 900, cost: 31.609941482543945, w: [-0.01892818], b: [-0.20579807]
step: 1000, cost: 23.057554244995117, w: [ 0.00169536], b: [-0.20529605]
step: 1100, cost: 16.91855239868164, w: [ 0.01916835], b: [-0.20486757]
step: 1200, cost: 12.511899948120117, w: [ 0.03397204], b: [-0.20450133]
step: 1300, cost: 9.348733901977539, w: [ 0.04651423], b: [-0.20418784]
st

In [9]:
# [] 데이터셋 분포 및
source = ColumnDataSource(
        data = dict(
            x = before_x,
            y = before_y,
            names=names,
            detail=labels,
        )
    )

hover = HoverTool(
        tooltips=[
            ("아파트", "@names"),
            ("매매가", "@detail"),
        ]
    )
TOOLS = "pan,box_zoom,reset"

p = figure(plot_width=900, plot_height=400, tools=[TOOLS,hover], title=chart_name)
p.scatter('x', 'y', size=6, source=source, alpha=0.4, fill_color="red", line_color="#ff9900")
p.circle(x=before_x, y=(sess.run(W)*before_x+sess.run(B)), line_width=3, line_cap="round")

show(p)

In [ ]:
# [] weight 에 따라 cost 0 을 수렴하는 과정, 그래프에는 GradientDescent로 한쪽면을 타고 내려가고 있음
# 사발 모양의 cost 그래프는 다음 주소 참고 http://pythonkim.tistory.com/11
p = figure(plot_width=300, plot_height=330, title=chart_name)
for hw, hc in zip(history['weight'], history['cost']):
    p.circle(x=hw, y=hc, line_width=3, line_cap="round")
show(p)

In [101]:
sess.close

<bound method Session.close of <tensorflow.python.client.session.Session object at 0x116b79048>>